In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
from iac_sketch import data, parse, sketch

In [ ]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(parse)
importlib.reload(sketch)
architect = sketch.Architect("../public/components")
registry = architect.parse()

In [ ]:
# Create a directed graph from the DataFrame
graph = nx.from_pandas_edgelist(
    registry["link"],
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.DiGraph,
)

# Visualize the graph
nx.draw(graph, with_labels=True)

In [ ]:
graph.add_nodes_from(registry["metadata"]["entity"].values)

In [ ]:
connected_components = [_ for _ in nx.connected_components(graph.to_undirected())]
metadata = registry["metadata"].set_index("entity")
metadata["connected_component_category"] = -1
for i, comps in enumerate(connected_components):
    metadata.loc[list(comps), "connected_component_category"] = i

In [ ]:
pair_connectivity = nx.all_pairs_node_connectivity(graph)

In [ ]:
sum(pair_connectivity["accept_request"].values()) > 0

In [ ]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

In [ ]:
# Find invalid testcases
registry.view(["testcase", "status"])